# Keras MNIST Model Deployment

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
pip install keras
```

## Train locally
 

In [ ]:
import numpy as np
import math
import datetime
#from seldon.pipeline import PipelineSaver
import os
import tensorflow as tf
from keras import backend
from keras.models import Model,load_model
from keras.layers import Dense,Input
from keras.layers import Dropout
from keras.layers import Flatten, Reshape
from keras.constraints import maxnorm
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D

from keras.callbacks import TensorBoard

class MnistFfnn(object):

    def __init__(self,
                 input_shape=(784,),
                 nb_labels=10,
                 optimizer='Adam',
                 run_dir='tensorboardlogs_test'):
        
        self.model_name='MnistFfnn'
        self.run_dir=run_dir
        self.input_shape=input_shape
        self.nb_labels=nb_labels
        self.optimizer=optimizer
        self.build_graph()

    def build_graph(self):
                            
        inp = Input(shape=self.input_shape,name='input_part')

        #keras layers
        with tf.name_scope('dense_1') as scope:
            h1 = Dense(256,
                         activation='relu',
                         W_constraint=maxnorm(3))(inp)
            drop1 = Dropout(0.2)(h1)

        with tf.name_scope('dense_2') as scope:
            h2 = Dense(128,
                       activation='relu',
                       W_constraint=maxnorm(3))(drop1)
            drop2 = Dropout(0.5)(h2)
            
            out = Dense(self.nb_labels,
                        activation='softmax')(drop2)

        self.model = Model(inp,out)
        
        if self.optimizer ==  'rmsprop':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='rmsprop',
                               metrics=['accuracy'])
        elif self.optimizer == 'Adam':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='Adam',
                               metrics=['accuracy'])
            
        print('graph builded')

    def fit(self,X,y=None,
            X_test=None,y_test=None,
            batch_size=128,
            nb_epochs=2,
            shuffle=True):
        
        now = datetime.datetime.now()
        tensorboard_logname = self.run_dir+'/{}_{}'.format(self.model_name,
                                                           now.strftime('%Y.%m.%d_%H.%M'))      
        tensorboard = TensorBoard(log_dir=tensorboard_logname)
        
        self.model.fit(X,y,
                       validation_data=(X_test,y_test),
                       callbacks=[tensorboard],
                       batch_size=batch_size, 
                       nb_epoch=nb_epochs,
                       shuffle = shuffle)
        return self
    
    def predict_proba(self,X):

        return self.model.predict_proba(X)
    
    def predict(self, X):
        probas = self.model.predict_proba(X)
        return([[p>0.5 for p in p1] for p1 in probas])
        
    def score(self, X, y=None):
        pass

    def get_class_id_map(self):
        return ["proba"]

class MnistConv(object):

    def __init__(self,
                 input_shape=(784,),
                 nb_labels=10,
                 optimizer='Adam',
                 run_dir='tensorboardlogs_test',
                 saved_model_file='MnistClassifier.h5'):
        
        self.model_name='MnistConv'
        self.run_dir=run_dir
        self.input_shape=input_shape
        self.nb_labels=nb_labels
        self.optimizer=optimizer
        self.saved_model_file=saved_model_file
        self.build_graph()

    def build_graph(self):
                                                                
        inp = Input(shape=self.input_shape,name='input_part')
        inp2 = Reshape((28,28,1))(inp)      
        #keras layers
        with tf.name_scope('conv') as scope:
            conv = Convolution2D(32, 3, 3,
                                 input_shape=(32, 32, 3),
                                 border_mode='same',
                                 activation='relu',
                                 W_constraint=maxnorm(3))(inp2)
            drop_conv = Dropout(0.2)(conv)
            max_pool = MaxPooling2D(pool_size=(2, 2))(drop_conv)

        with tf.name_scope('dense') as scope:
            flat = Flatten()(max_pool)                
            dense = Dense(128,
                          activation='relu',
                          W_constraint=maxnorm(3))(flat)
            drop_dense = Dropout(0.5)(dense)
            
            out = Dense(self.nb_labels,
                        activation='softmax')(drop_dense)

        self.model = Model(inp,out)
        
        if self.optimizer ==  'rmsprop':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='rmsprop',
                               metrics=['accuracy'])
        elif self.optimizer == 'Adam':
            self.model.compile(loss='categorical_crossentropy',
                               optimizer='Adam',
                               metrics=['accuracy'])
            
        print('graph builded')

    def fit(self,X,y=None,
            X_test=None,y_test=None,
            batch_size=128,
            nb_epochs=2,
            shuffle=True):
        
        now = datetime.datetime.now()
        tensorboard_logname = self.run_dir+'/{}_{}'.format(self.model_name,
                                                           now.strftime('%Y.%m.%d_%H.%M'))      
        tensorboard = TensorBoard(log_dir=tensorboard_logname)
        
        self.model.fit(X,y,
                       validation_data=(X_test,y_test),
                       callbacks=[tensorboard],
                       batch_size=batch_size, 
                       nb_epoch=nb_epochs,
                       shuffle = shuffle)
        #if not os.path.exists('saved_model'):
        #    os.makedirs('saved_model')
        self.model.save(self.saved_model_file)
        return self
    
    def predict_proba(self,X):
        return self.model.predict_proba(X)
    
    def predict(self, X):
        probas = self.model.predict_proba(X)
        return([[p>0.5 for p in p1] for p1 in probas])
        
    def score(self, X, y=None):
        pass

    def get_class_id_map(self):
        return ["proba"]



In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('data/MNIST_data', one_hot=True)
X_train = mnist.train.images
y_train = mnist.train.labels
X_test = mnist.test.images
y_test = mnist.test.labels
mc = MnistConv()
mc.fit(X_train,y=y_train,
    X_test=X_test,y_test=y_test)



Wrap model using s2i

In [ ]:
!s2i build . seldonio/seldon-core-s2i-python3:0.7 keras-mnist:0.1

In [ ]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 keras-mnist:0.1

Send some random features that conform to the contract

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm mnist_predictor --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python3:0.7 keras-mnist:0.1

In [ ]:
!kubectl create -f keras_mnist_deployment.json

In [ ]:
!kubectl rollout status deploy/keras-mnist-deployment-keras-mnist-predictor-8baf5cc

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete